# Represent ORACC Texts in UTF-8 Cuneiform
The code in this notebook will parse [ORACC](http://oracc.org) `JSON` files to extract Akkadian text in lemmatizations, transliterations and cuneiform from one or more [ORACC](http://oracc.org) projects. 

In [1]:
import pandas as pd
import zipfile
import json
import tqdm
import os
import sys
import pickle
import re
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output', 'corpus']
make_dirs(directories)

## 1.1 Input Project Names
Provide a list of one or more project names, separated by commas. Note that subprojects must be listed separately, they are not included in the main project. For instance:

blms, akklove, cams/anzu, cams/barutu, cams/gkab, cams/selbi, ccpo, cmawro, cmawro/cmawr2, cmawro/maqlu, dcclt, dcclt/nineveh, dcclt/signlists, glass, hbtin, riao, ribo/babylon0, ribo/babylon1, ribo/babylon2, ribo/babylon3, ribo/babylon4, ribo/babylon5, ribo/babylon6, ribo/babylon7, ribo/babylon8, ribo/babylon10, rimanum, rinap/rinap1, rinap/rinap3, rinap/rinap4, rinap/rinap5, saao/saa01, saao/saa02, saao/saa03, saao/saa04, saao/saa05, saao/saa06, saao/saa07, saao/saa08, saao/saa09, saao/saa10, saao/saa11, saao/saa12, saao/saa13, saao/saa14, saao/saa15, saao/saa16, saao/saa17, saao/saa18, saao/saa19, saao/saa20, saao/saa21, suhu

In [3]:
projects = input('Project(s): ').lower()

Project(s): blms, akklove, cams/anzu, cams/barutu, cams/gkab, cams/selbi, ccpo, cmawro, cmawro/cmawr2, cmawro/maqlu, dcclt, dcclt/nineveh, dcclt/signlists, glass, hbtin, riao, ribo/babylon0, ribo/babylon1, ribo/babylon2, ribo/babylon3, ribo/babylon4, ribo/babylon5, ribo/babylon6, ribo/babylon7, ribo/babylon8, ribo/babylon10, rimanum, rinap/rinap1, rinap/rinap3, rinap/rinap4, rinap/rinap5, saao/saa01, saao/saa02, saao/saa03, saao/saa04, saao/saa05, saao/saa06, saao/saa07, saao/saa08, saao/saa09, saao/saa10, saao/saa11, saao/saa12, saao/saa13, saao/saa14, saao/saa15, saao/saa16, saao/saa17, saao/saa18, saao/saa19, saao/saa20, saao/saa21, suhu


## 1.2 Split the List of Projects and Download the ZIP files.
Split the list of projects and create a list of project names, using the `format_project_list()` and `oracc_download()` functions in the `utils` module. The code of these functions is discussed in more detail in 2.1.0. Download ORACC JSON Files.

In [4]:
p = format_project_list(projects)
oracc_download(p)

## <a name="head21"></a>2.1 The `parsejson()` function
The `parsejson()` function will "dig into" the `json` file (transformed into a dictionary) until it finds the relevant data. The `json` file consists of a hierarchy of `cdl` nodes; only the lowest nodes contain lemmatization data. The function goes down this hierarchy by calling itself when another `cdl` node is encountered. For nore information about the data hierarchy in the [ORACC](http://oracc.org) `json` files, see [ORACC Open Data](http://oracc.museum.upenn.edu/doc/opendata/index.html).

The argument of the `parsejson()` function is a `JSON` object, a dictionary that initially contains the entire contents of the original JSON file. The code takes the key `cdl` which itself contains an array (a list) of `JSON` objects. Iterating through these objects, if an object contains another `cdl` node, the function calls itself with this object as first argument. This way the function digs deeper and deeper into the `JSON` tree, until it does not encounter a `cdl` key anymore. Here we are at the level of individual words. The code checks for a key `f`, if it exists the signs are in the node `gdl` within the `f` node. 

In [7]:
def parsejson_signs(text):
    for JSONobject in text["cdl"]:
        field = ''
        if "cdl" in JSONobject: 
            parsejson_signs(JSONobject)
        if "type" in JSONobject and JSONobject["type"] == "field-start":
            field = JSONobject["subtype"]
        if "f" in JSONobject and not field in ['sg', 'pr']: # skip the fields "sign" and "pronunciation"
                                # in lexical texts
            lang = JSONobject["f"]["lang"]     #[:3] == "akk": #only Akkadian and Akkadian dialects
            word = JSONobject["f"]
            f = word["form"]
            if "sexified" in word["gdl"][0]:
                f = word["gdl"][0]["sexified"]
            if "cf" in word:
                if 'pos' in word:  #for some reason some words appear without pos. Provisionally treated as Noun
                    lemm = word["cf"] + '[' + word["gw"] + "]" + word["pos"]
                else:
                    lemm = word["cf"] + '[' + word["gw"] + "]N"
                lemm = lemm.replace(' ', '-') # remove commas and spaces from lemm
                lemm = lemm.replace(',', '')
            else:
                lemm = word["form"] # if word is unlemmatized
            all_.append(f)
            lemm_.append(lemm)
            lang_.append(lang)
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file is located in the directory `jsonzip`, named PROJECT.zip. 

Each of these files is extracted from the `zip` file and read with the command `json.loads()`, which reads the json data and transforms it into a Python dictionary (a sequence of keys and values).

This dictionary, which is called `text` is now sent to the `parsejson()` function. The function adds signs to the `sign_l` list.

In [8]:
all_ = []
lemm_ = []
ids_ = []
lang_ = []
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    files = z.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in tqdm.tqdm(files):                            #iterate over the file names
        id_no = filename[-13:-5]
        if id_no in ids_ and not "X" in id_no: # Check if P/Q number is already in there
            continue        # a text may appear in multiple projects
        id_text = project + id_no # id_text is, for instance, blms/P414332
        try:
            text = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
            data_json = json.loads(text)                # make it into a json object (essentially a dictionary)
            all_.append('Start'+id_text)
            lemm_.append('Start'+id_text)   # to keep all_ and lemm_ same length
            lang_.append('Start'+id_text)
            parsejson_signs(data_json)
            ids_.append(id_no)
            #print(filename)
        except:
            print(id_text + ' is not available or not complete')

  9%|███▋                                     | 36/395 [00:00<00:04, 86.55it/s]

blms/P357131 is not available or not complete


 31%|████████████▎                           | 122/395 [00:01<00:03, 87.08it/s]

blms/P384976 is not available or not complete


 64%|████████████████████████▉              | 252/395 [00:02<00:01, 117.33it/s]

...

 95%|█████████████████████████████████████▏ | 457/479 [00:02<00:00, 120.68it/s]

saao/saa14/P335196 is not available or not complete
saao/saa14/P335154 is not available or not complete


100%|██████████████████████████████████████████| 55/55 [00:02<00:00, 27.26it/s]


jsonzip/saao-saa21.zip does not exist or is not a proper ZIP file


100%|██████████████████████████████████████████| 33/33 [00:00<00:00, 81.80it/s]


## 3 Data Structuring
### 3.1 Transform the Data into a DataFrame


In [9]:
words_l = []
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']
operators = ['&', '%', '@', '×']
for e in tqdm.tqdm(all_):
    word = []
    if '1(šar₂{gal})' in e: # this cheating but it seems to work (appears in SKL 38)
            e = e.replace('1(šar₂{gal})', '1(šar₂)-gal')
    for s in separators: # first split word into signs   
        e = e.replace(s, ' ').strip()
    s_l = e.split()
    for sign in s_l:
        if sign[0].isdigit(): # 1(geš₂), 2(DIŠ), etc.
            sign = sign.lower()
        elif sign[-1] == ')': # qualified sign - get only the qualifier
            stack = []  # |GIŠ×(GIŠ%GIŠ)|(LAK277) becomes LAK277
            ind = {}    # LAK277(|GIŠ×(GIŠ%GIŠ)|) becomes |GIŠ×(GIŠ%GIŠ)|
            for i, c in reversed(list(enumerate(sign))):
                if c == ')':
                    stack.append(i)
                if c == '(':
                    ind[stack.pop()] = i   # find the opening parens that belongs to the closing parens at position -1    
            start = ind[len(sign)-1]   # this line fails on 1(šar₂{gal}) in SKL.
            t = sign[start+1:-1]
            if t.isupper(): #leave 1(diš) etc. alone
                sign = t
            
        if '|' in sign:  # separate |DU.DU| and |DU+DU| into its components but not |DU&DU|
                        # and also not |DU.DU&DU|
            flag = False
            for o in operators:
                if o in sign:
                    flag = True
            if not flag:
                for s in separators2:
                    sign = sign.replace(s, ' ').strip() 
                sign_l = sign.split()
                word.extend(sign_l)
                continue
        elif "+" in sign:  # + as marker of gloss
            sign = sign.replace('+', ' ').strip()
            sign_l = sign.split()
            word.extend(sign_l)
            continue
        word.append(sign)
    words_l.append(word)           

100%|█████████████████████████████| 1541908/1541908 [00:18<00:00, 83506.24it/s]


In [10]:
with open("output/ogsl.p", "rb") as f:
    o = pd.read_pickle(f)

In [11]:
val = list(o["value"])
utf = list(o["utf8"])
names = list(o["name"])

In [12]:
d = dict(zip(names, utf))
d2 = dict(zip(val,names))

In [13]:
names_l = []
utf8_l = []
for w in tqdm.tqdm(words_l):
    seq = [d2[s.lower()] if s.lower() in d2 else s for s in w]
    names_l.append(seq)
    utf8 = [d[n] if n in d else n for n in seq]
    utf8_l.append(''.join(utf8))

100%|████████████████████████████| 1541908/1541908 [00:14<00:00, 104965.49it/s]


In [14]:
df = pd.DataFrame({"transliteration":all_, "words":words_l, "names":names_l, "utf-8":utf8_l, "lemm" : lemm_, "lang" : lang_})
df

,transliteration,words,names,utf-8,lemm,lang
0,Startblms/P414332,[Startblms/P414332],[Startblms/P414332],Startblms/P414332,Startblms/P414332,Startblms/P414332
1,x-x,"[x, x]","[X, X]",XX,x-x,sux
2,dam-ŋu₁₀,"[dam, ŋu₁₀]","[DAM, MU]",𒁮𒈬,dam[spouse]N,sux
3,mu-ni-ib₂-be₂,"[mu, ni, ib₂, be₂]","[MU, NI, TUM, BI]",𒈬𒉌𒌈𒁉,e[speak]V/t,sux
4,x-ri,"[x, ri]","[X, RI]",X𒊑,x-ri,akk-x-stdbab
5,mu-ti-ma,"[mu, ti, ma]","[MU, TI, MA]",𒈬𒋾𒈠,mutu[husband]N,akk-x-stdbab
6,i-qab-bi,"[i, qab, bi]","[I, GABA, BI]",𒄿𒃮𒁉,qabû[say]V,akk-x-stdbab
7,x-x,"[x, x]","[X, X]",XX,x-x,sux
8,dumu-ŋu₁₀,"[dumu, ŋu₁₀]","[TUR, MU]",𒌉𒈬,dumu[child]N,sux
9,mu-ni-ib₂-be₂,"[mu, ni, ib₂, be₂]","[MU, NI, TUM, BI]",𒈬𒉌𒌈𒁉,e[speak]V/t,sux


In [15]:
with open("corpus/all_df.p", "wb") as w:
    pickle.dump(df, w)

# Save as Text
Save three different representations of the Akkadian text. Each representation is saved in a separate text file:
- in transliteration        ===> akk_tl.txt
- in lemmatized format   ===> akk_lemm.txt
- in cuneiform (utf-8)    ===> akk_utf8.txt

In [17]:
df_akk = df[df["lang"].str[:3].isin(["akk", "Sta"])]
rep_d = {"akk_utf8": "utf-8", "akk_lemm": "lemm", "akk_tl" : "transliteration"}
for rep in rep_d:
    text = ' '.join(df_akk[rep_d[rep]]).strip()
    text = text.replace(' Start', '\n').strip()
    text = text.replace('Start', '')
    text = re.sub(r'\n+', '\n', text)
    file = "corpus/" + rep + ".txt"
    with open(file, 'w', encoding="utf-8") as w:
        w.write(text)